In [1]:
import os
import csv

samples=[]
with open('./data/driving_log.csv') as csvfile:
    reader=csv.reader(csvfile)
    for line in reader:
        samples.append(line)

In [2]:
from sklearn.model_selection import train_test_split
train_samples,validation_samples=train_test_split(samples,test_size=0.2)

In [3]:
import cv2
import numpy as np
import sklearn

def generator(samples,batch_size=32):
    num_samples=len(samples)
    while 1:
#         shuffle(samples)
        for offset in range(0,num_samples,batch_size):
            batch_samples=samples[offset:offset+batch_size]
            
            images=[]
            angles=[]
            for batch_sample in batch_samples:
                name=batch_sample[0].split('/')[-1]
                center_image=cv2.imread(name)
                center_angle=float(batch_sample[3])
                images.append(center_image)
                angles.append(center_angle)
            
            X_train=np.array(images)
            y_train=np.array(angles)
            yield sklearn.utils.shuffle(X_train,y_train)
            

In [4]:
train_generator=generator(train_samples,batch_size=32)
validation_generator=generator(validation_samples,batch_size=32)

In [8]:
from keras.models import Sequential
from keras.layers import Dense,Conv2D,Flatten,Cropping2D,BatchNormalization,Activation,Dropout,MaxPooling2D,Lambda

from keras.optimizers import Adam
from keras.regularizers import l2
from keras.callbacks import EarlyStopping, ModelCheckpoint

In [10]:
ch, row, col = 3, 160, 320
model=Sequential()
model.add(Lambda(lambda x: x/255.0 - 0.5,
        input_shape=(row, col,ch)))

model.add(Cropping2D(cropping=((70,25),(0,0))))

model.add(Conv2D(64,(5,5),padding='same',kernel_regularizer=None))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(MaxPooling2D(pool_size=(3,3)))

model.add(Conv2D(128,(5,5),padding='same',kernel_regularizer=None))
model.add(Activation('elu'))
model.add(MaxPooling2D(pool_size=(3,3)))

model.add(Conv2D(256,(5,5),padding='same',kernel_regularizer=None))
model.add(Activation('elu'))
model.add(Dropout(0.5))
model.add(MaxPooling2D(pool_size=(3,3)))
model.add(Flatten())

model.add(Dense(512,kernel_regularizer=None))
model.add(Activation('elu'))

model.add(Dense(256,kernel_regularizer=None))
model.add(Activation('elu'))

model.add(Dense(1))
model.summary()

Instructions for updating:
keep_dims is deprecated, use keepdims instead
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lambda_1 (Lambda)            (None, 160, 320, 3)       0         
_________________________________________________________________
cropping2d_1 (Cropping2D)    (None, 65, 320, 3)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 65, 320, 64)       4864      
_________________________________________________________________
activation_1 (Activation)    (None, 65, 320, 64)       0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 65, 320, 64)       256       
_________________________________________________________________
dropout_1 (Dropout)          (None, 65, 320, 64)       0         
_________________________________________________________________
max

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0.0, patience=2)
checkpointer = ModelCheckpoint(filepath='model-{val_loss:.5f}.h5', verbose=1, save_best_only=True)
model.compile(loss='mse', optimizer='adam')
model.fit_generator(train_generator, steps_per_epoch=len(train_samples),
                    validation_data=validation_generator,
            validation_steps=len(validation_samples), nb_epoch=5,callbacks=[early_stopping,checkpointer],verbose=1)
model.save('model.h5')

Instructions for updating:
keep_dims is deprecated, use keepdims instead


c:\programdata\anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: The semantics of the Keras 2 argument `steps_per_epoch` is not the same as the Keras 1 argument `samples_per_epoch`. `steps_per_epoch` is the number of batches to draw from the generator at each epoch. Basically steps_per_epoch = samples_per_epoch/batch_size. Similarly `nb_val_samples`->`validation_steps` and `val_samples`->`steps` arguments have changed. Update your method calls accordingly.
  
c:\programdata\anaconda3\lib\site-packages\ipykernel_launcher.py:6: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., steps_per_epoch=2036, validation_data=<generator..., validation_steps=510, callbacks=[<keras.ca..., verbose=1, epochs=5)`
  


Epoch 1/5
2036/2036 [==============================] - 630s 310ms/step - loss: 4.5274 - val_loss: 45.7511
Epoch 2/5
 683/2036 [=========>....................] - ETA: 6:06 - loss: 0.0162